##### Prueba BI

### Objetivo de la prueba

El objetivo de la prueba de código es validar tus conocimientos en la extracción, transformación y limpieza de los datos, a la vez que obtienes conclusiones lógicas sobre estos. Para ello, tendrás que realizar el análisis que te comentamos a continuación.


### Instrucciones de la prueba

En la carpeta "data" del directorio raíz, encontrarás varias fuentes de datos con las que trabajarás a lo largo de la prueba.

- "finance.db": Base de datos SQLITE a la que accederás únicamente a través de una librería de python.
- "finance_data_1.csv" y "finance_data_2.csv".
- "finance_data.json" y "funny_finance_data.json".

El lenguaje utilizado en la prueba es Python.

Añade las celdas que sean necesaria y saca los análisis, conclusiones o explicaciones oportunas. Es de agradecer que vayas añadiendo comentarios en cada paso que realizas durante la prueba. Intenta justificar y mostrar por qué haces las cosas.

Recuerda también que este archivo deberá poder ser ejecutado, así que antes de enviarlo comprueba que todas las celdas funcionan.

La primera parte de la prueba consiste en un conjunto de consultas de SQL. Esta parte está resuelta y no hay que hacer nada más allá de quizás cambiar las / por \\\\ en función del sistema operativo que utilices.

A continuación, te indicamos los paquetes y conexiones a realizar para comenzar la prueba.

In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect("data/finance.db")

La tabla "loans_data" contiene información de las "particiones" de Agosto y Septiembre de 2018 de un servicio de contratación de préstamos. Contiene los siguientes campos:

- id: identificador de oferta de financiación no nulo.
- operationdate: fecha de operación en unix_timestamp_epoch.
- userid: identificador de usuario.
- tin: tipo de interés nominal
- status: status_1 (inicial), status_2 (intermedio), KO (Denegado), OK (Admitido), ABANDONO (Desistido)...
- rand_score: puntuación de riesgo. A mayor puntuación, mayor riesgo.
- isinasnef: flag de pertenencia a la lista Asnef.
- type: tipo de financiación
- month_partition: partición de mes


# Parte 1: Análisis de la tabla "loans_data"

Mediante consultas SQL (añadiendo en las respuestas las queries realizadas) trata de responder a las siguientes preguntas. Añade por favor cualquier comentario adicional.

#### 1.1. ¿Cuántos usuarios hay?

In [3]:
query1 = ("""
          SELECT COUNT(DISTINCT userid) as n_usuarios
          FROM loans_data
          """
         )
       
n_usuarios = pd.read_sql_query(query1, conn)
print(f"Hay un total de {n_usuarios.iloc[0,0]} usuarios")

Hay un total de 6317 usuarios


#### 1.2. ¿Cuál es el tin medio por tipo de financiación?

In [4]:
query2 = ("""
          SELECT type, AVG(tin) as tin_medio
          FROM loans_data
          GROUP BY type
          """
         )

tin_medio = pd.read_sql_query(query2, conn)
for _, row in tin_medio.iterrows():
    print(f"El tin_medio por tipo de financiación {row['type']} es {row['tin_medio']}")

El tin_medio por tipo de financiación type_1 es 16.505683371979362
El tin_medio por tipo de financiación type_3 es 10.620689655172415


#### 1.3. ¿Cuántos usuarios tienen una puntuación entre 0.01 y 0.015?

In [5]:
query3 = ("""
          SELECT COUNT(DISTINCT userid)
          FROM loans_data
          WHERE rand_score BETWEEN 0.01 AND 0.015
          """
         )

n_usuarios_001_0015 = pd.read_sql_query(query3, conn)
print(f"Hay un total de {n_usuarios_001_0015.iloc[0,0]} usuarios con una puntuación entre 0,01 y 0,015")

Hay un total de 1884 usuarios con una puntuación entre 0,01 y 0,015


#### 1.4. ¿Cuál es el último estado, tin y tipo de financiación de los siguientes usuarios?
- 33bd9523dde697a59905cb31736d75ea 
- 00014d1ac1ce489bba5f6cdb8d7b6112?

In [6]:
query4 = ("""
          SELECT T1.userid, status, tin, type
          FROM loans_data T1
          INNER JOIN (SELECT userid, MAX(operationdate) as operationdate
                      FROM loans_data
                      WHERE userid = "33bd9523dde697a59905cb31736d75ea" OR userid = "00014d1ac1ce489bba5f6cdb8d7b6112"
                      GROUP BY userid) sq
          ON T1.operationdate = sq.operationdate
          """
         )

last_state = pd.read_sql_query(query4, conn)
print(last_state)

                             userid           status    tin    type
0  00014d1ac1ce489bba5f6cdb8d7b6112        status_KO  26.28  type_1
1  33bd9523dde697a59905cb31736d75ea  status_ABANDONO  24.00  type_1


#### 1.5. ¿Cuál es la tasa de aprobación en este periodo?

In [7]:
query5 = ("""
          SELECT status, count(*) * 100.0 / (SELECT count(*)
                                             FROM loans_data) AS app_por
          FROM loans_data
          WHERE status = "status_OK"
          """
         )


tasa = pd.read_sql_query(query5, conn)
print(f"La tasa de aprobación en el período es: {tasa.iloc[0,1]}")

La tasa de aprobación en el período es: 20.498516320474778


# Parte 2: Unión y limpieza de datos de los ficheros .csv

En el mismo directorio del notebook y la base de datos que has utilizado, se encuentran dos ficheros en formato csv con datos adicionales ("finance_data_1.csv" y "finance_data_2.csv"). El objetivo de este apartado es que obtengas en un objeto **dataframe de la libreria "pandas"** todos los datos de las fuentes mencionadas, en **un solo dataframe**. Es decir, de ambos ficheros csv y de los datos de la parte 1.

A tu criterio queda la limpieza de los datos, el tipo de dato de cada variable, la forma de obtener los datos de cada fuente, la forma de unirlos, el resultado final del dataframe...

Por favor, añade todos los pasos que realices en el notebook, junto con los comentarios que consideres oportunos.

# Parte 3: Análisis de ficheros json

Una vez tienes todos los datos en la nueva tabla y en un dataframe juntos, tenemos datos adicionales de algunos usuarios en otra fuente de datos distinta. En este caso son dos archivos json. Se llaman "finance_data.json" y "funny_finance_data.json".

Importa los datos de los archivos JSON, únelos entre sí y relacionalo con el dataframe obtenidos en la parte 2. Después responde a las siguientes preguntas.

#### 3.1. ¿Cuántos usuarios de Valencia han sido denegados?

#### 3.2. ¿Cuál es la media de edad para los préstamos aceptados?

#### 3.3. Por lo general, ¿Acceden a un TIN más bajo los hombres o las mujeres?

#### 3.4. ¿Cual es la tendencia de la tasa de aprobación en el tiempo (por días) y qué factores crees que han influido?

# Parte 4: Conclusiones sobre los datos obtenidos 

Trata de sacar un insight/conclusión de cualquier dataframe de los que has ido sacando o de la tabla con la que has comenzado la prueba y realiza al menos una visualización con cualquier librería de visualización con la que te sientas cómodo (matplotlib, bokeh...). Comenta los pasos que sigues para ello.